In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('ML-lib').getOrCreate()

In [5]:
training = spark.read.csv('test2.csv', header = True, inferSchema = True)
training.show()

+----+---+----------+------+
|name|age|experience|salary|
+----+---+----------+------+
| ale| 31|        10| 30000|
| axe| 30|         8| 25000|
| dke| 29|         4| 20000|
| dsf| 24|         3| 20000|
| fdf| 21|         1| 15000|
| vdv| 23|         2| 18000|
+----+---+----------+------+



In [6]:
training.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [7]:
training.columns

['name', 'age', 'experience', 'salary']

In [9]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols= ['age', 'experience'], outputCol='independent feature')

In [10]:
output = featureAssembler.transform(training)

In [11]:
output.show()

+----+---+----------+------+-------------------+
|name|age|experience|salary|independent feature|
+----+---+----------+------+-------------------+
| ale| 31|        10| 30000|        [31.0,10.0]|
| axe| 30|         8| 25000|         [30.0,8.0]|
| dke| 29|         4| 20000|         [29.0,4.0]|
| dsf| 24|         3| 20000|         [24.0,3.0]|
| fdf| 21|         1| 15000|         [21.0,1.0]|
| vdv| 23|         2| 18000|         [23.0,2.0]|
+----+---+----------+------+-------------------+



In [12]:
output.columns

['name', 'age', 'experience', 'salary', 'independent feature']

In [13]:
finalizedData = output.select('independent feature', 'salary')

In [14]:
finalizedData.show()

+-------------------+------+
|independent feature|salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [25]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = finalizedData.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'independent feature', labelCol = 'salary')
model = regressor.fit(train_data)

In [26]:
model.coefficients 

DenseVector([-90.9091, 1272.7273])

In [27]:
model.intercept

17545.454545455385

In [31]:
pred_results = model.evaluate(test_data)
pred_results.predictions.show()

+-------------------+------+------------------+
|independent feature|salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|16909.090909091006|
|         [24.0,3.0]| 20000|19181.818181818242|
|        [31.0,10.0]| 30000|27454.545454545525|
+-------------------+------+------------------+



In [33]:
pred_results.meanSquaredError

3597796.1432506596